In [5]:
import psycopg2 as pg
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.tools as tls

In [6]:
connection = pg.connect("dbname=keeper user=jeff")

In [7]:
trace_df = pd.read_sql("select * from action_trace where action_id = 409", con=connection)
twist_df = trace_df.pivot(index='millis', columns='measure', values='value')
if twist_df.actual_vel.mean() < 0:
    twist_df.actual_vel = twist_df.actual_vel * -1
twist_df.setpoint_vel = twist_df.setpoint_vel / 10.0
twist_df.profile_vel = twist_df.profile_vel / 10.0
twist_df['ticks_error'] = twist_df.actual_ticks - twist_df.profile_ticks

In [30]:
layout1={'title': 'Velocity',
        'xaxis': {'title': 'milliseconds'},
        'yaxis': {'title': 'ticks/100ms'},
        'yaxis2': {'title': 'amps', 'side': 'right', 'overlaying': 'y'},
        'height': 600
       }
f1 = go.FigureWidget(layout=layout1)
f1.add_trace(go.Scatter(name='profile', x=twist_df.index, y=twist_df.profile_vel))
f1.add_trace(go.Scatter(name='setpoint',x=twist_df.index, y=twist_df.setpoint_vel))
f1.add_trace(go.Scatter(name='actual',x=twist_df.index, y=twist_df.actual_vel))
f1.add_trace(go.Scatter(name='current',x=twist_df.index, y=twist_df.drive_current, line={'color': 'rgba(22, 12, 204, 0.1)'}, yaxis='y2'))
f1

FigureWidget({
    'data': [{'name': 'profile',
              'type': 'scatter',
              'uid': '81ff50e…

In [32]:
layout2={'title': 'Distance Error and Gyro Angle',
        'xaxis': {'title': 'milliseconds'},
        'yaxis': {'title': 'ticks'},
        'yaxis2': {'title': 'degrees', 'side': 'right', 'overlaying': 'y'},
         'height': 600
       }
f2 = go.FigureWidget(layout=layout2)
f2.add_trace(go.Scatter(name='distance', x=twist_df.index, y=twist_df.ticks_error))
f2.add_trace(go.Scatter(name='gyro',x=twist_df.index, y=twist_df.gyro_angle, line={'color': 'rgba(204, 12, 22, 1)'}, yaxis='y2'))
f2

FigureWidget({
    'data': [{'name': 'distance',
              'type': 'scatter',
              'uid': '3a9d16…